# Unpacking of Raw ENC Data

Assumed directory structure:
```
.
├── 00_raw_data
├── 01_unpacked_data
├── 02_processed_data
├── 03_marked_for_qa
├── 04_qa_reviewed
├── 05_final_for_export
├── 01_unpacking.ipynb
├── 02_processing.ipynb
├── 02_standardize.ipynb
├── 03_deduplicate.ipynb
├── 04_postprocessing.ipynb
└── ...
```

In [ ]:
data_file_name = "data.csv" # Name of raw data file without "raw_" prefix
bucket_name = "enc-bucket_name" #TODO

## Imports

In [ ]:
import pandas as pd
from itables import show
import re

## fixed parameters

In [ ]:
file_path = "00_raw_data"
upload_path = "01_unpacked_data"

## Init GCloud I/O

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket(bucket_name)

## Download Raw Data

In [ ]:
raw_blob_name = f"{file_path}/raw_{data_file_name}"

In [ ]:
bucket.blob(raw_blob_name).download_to_filename(raw_blob_name)
print(f"Downloaded {raw_blob_name}")

In [ ]:
raw_data = pd.read_csv(raw_blob_name)

## Unpack

In [ ]:
from aroa_etl.enc.unpacking import unpack

In [ ]:
show(raw_data)

In [ ]:
unpacked_data = unpack(raw_data,"json_data", additional_splits_on=lambda col: re.search(r"(category)",col), split_re=r"[\|;,\s]")

In [ ]:
show(unpacked_data)

## Upload unpacked data

In [ ]:
unpacked_file_name = f"{upload_path}/unpacked_{data_file_name}"

In [ ]:
unpacked_data.to_csv(unpacked_file_name,sep="|")

In [ ]:
blob = bucket.blob(unpacked_file_name)
blob.upload_from_filename(unpacked_file_name)